In [1]:
# Monitor SC

In [2]:
!pip install matplotlib seaborn pandas

In [3]:
import logging
import uuid

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import smartpynector as sp
from utils import *

In [4]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("MonitorSC")

In [5]:
# Constants
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
# prefixes of ontologies used in the graph patterns
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
    "ex": "http://example.org/",
}
# graph pattern describing a timeseries of temperature measurements with timestamp between ?startTimestamp and ?endTimestamp
TEMP_TIMESERIES_GRAPH_PATTERN = """?timeseries rdf:type ex:Timeseries .
                            ?timeseries ex:hasMeasurement ?meas .
                            ?timeseries ex:measuredAfter ?startTimestamp .
                            ?timeseries ex:measuredBefore ?endTimestamp .
                            ?meas rdf:type saref:Measurement .
                            ?meas saref:isMeasuredIn saref:TemperatureUnit .
                            ?meas saref:hasValue ?temp .
                            ?meas saref:hasTimestamp ?timestamp .
                            """

# graph pattern describing a timeseries of humidity measurements with timestamp between ?startTimestamp and ?endTimestamp
HUMIDITY_TIMESERIES_GRAPH_PATTERN = """?timeseries rdf:type ex:Timeseries .
                            ?timeseries ex:hasMeasurement ?meas .
                            ?timeseries ex:measuredAfter ?startTimestamp .
                            ?timeseries ex:measuredBefore ?endTimestamp .
                            ?meas rdf:type saref:Measurement .
                            ?meas saref:isMeasuredIn saref:HumidityUnit .
                            ?meas saref:hasValue ?humidity .
                            ?meas saref:hasTimestamp ?timestamp .
                            """

In [6]:
def start_monitor_kb(kb_id, kb_name, kb_description, ke_endpoint):
    # delete in case allready exists
    delete_knowledge_base(kb_id, ke_endpoint)

    # register kb
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    global ask_temp_measurements_ki
    ask_temp_measurements_ki = register_ask_knowledge_interaction(
        TEMP_TIMESERIES_GRAPH_PATTERN,
        "ask-temp-historical-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    global ask_humidity_measurements_ki
    ask_humidity_measurements_ki = register_ask_knowledge_interaction(
        HUMIDITY_TIMESERIES_GRAPH_PATTERN,
        "ask-humidity-historical-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

In [7]:
start_monitor_kb(
    "http://example.org/monitor",
    "Monitor",
    "Monitor for storage",
    "http://knowledge_engine:8280/rest/",
)

2023-06-25 15:33:14 INFO deleted http://example.org/monitor
2023-06-25 15:33:14 INFO registered Monitor
2023-06-25 15:33:15 INFO received issued knowledge interaction id: http://example.org/monitor/interaction/ask-temp-historical-measurements
2023-06-25 15:33:15 INFO received issued knowledge interaction id: http://example.org/monitor/interaction/ask-humidity-historical-measurements


In [8]:
def plot_data(data, meas_type: str):
    meas_types = ["temp", "humidity"]
    if data is None:
        raise ValueError("Data cannot be None")
    if meas_type not in meas_types:
        raise ValueError(f"meas_type has to be one of {meas_types}")

    # Convert data to a DataFrame
    df = pd.DataFrame(data)

    # Convert timestamp and temperature columns to appropriate data types
    df["timestamp"] = df["timestamp"].str.strip('"')
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    df[meas_type] = df[meas_type].str.strip('"')
    df[meas_type] = pd.to_numeric(df[meas_type])

    # Set figure size
    plt.figure(figsize=(24, 6))

    # Create line plot using seaborn
    sns.lineplot(data=df, x="timestamp", y=meas_type)
    plt.xticks(rotation=90)

    # Show plot
    plt.show()

In [9]:
def perform_ask(ask_ki_id, start_time, end_time):
    historical_measurements = ask(
        [
            {
                "timeseries": f"<{THERMOSTAT_API_URL}/timeseries/{str(uuid.uuid4())}>",
                "startTimestamp": f'"{start_time}"',
                "endTimestamp": f'"{end_time}"',
            }
        ],
        ask_ki_id,
        "http://example.org/monitor",
        "http://knowledge_engine:8280/rest/",
    )

    # logger.info(f"Historical measurements: {historical_measurements}")
    if historical_measurements and historical_measurements != [{}]:
        return historical_measurements
    else:
        logger.info("No historical measurements found.")
        return None

In [10]:
from datetime import datetime, timezone

from IPython.display import display
from ipywidgets import Button, DatetimePicker, HBox, Output, VBox


def convert_datetimes(start_widget, end_widget):
    # Convert the datetime object from UTC to the local timezone
    start_datetime_obj = (
        start_widget.value.astimezone().replace(microsecond=0).isoformat()
    )
    end_datetime_obj = end_widget.value.astimezone().replace(microsecond=0).isoformat()
    return start_datetime_obj, end_datetime_obj


start_widget = DatetimePicker(description="Start Time:", disabled=False)
start_widget.value = datetime.now(timezone.utc).replace(microsecond=0)

end_widget = DatetimePicker(description="End Time:", disabled=False)
end_widget.value = datetime.now(timezone.utc).replace(microsecond=0)

submit_button = Button(description="Submit", button_style="danger")

out = Output()


def on_button_click(button):
    with out:
        out.clear_output()
        start_timestamp, end_timestamp = convert_datetimes(start_widget, end_widget)
        logger.info(f"Start: {start_timestamp}, End: {end_timestamp}")
        temp_data = perform_ask(
            ask_temp_measurements_ki, start_timestamp, end_timestamp
        )
        humidity_data = perform_ask(
            ask_humidity_measurements_ki, start_timestamp, end_timestamp
        )
        plot_data(temp_data, "temp")
        plot_data(humidity_data, "humidity")


submit_button.on_click(on_button_click)

display(VBox([HBox([start_widget, end_widget, submit_button]), out]))